# **Atividade 02 - Visão Computacional e Percepção** 

\\

---

**Alunos:**
- Glenda Proença Train
- Jhoser Alaff dos Santos Matheus

\\

---
**Especificação:**

* Pesquise sobre calibração de câmera.

* Leia os tutoriais de calibração do openCV: 
  * https://docs.opencv.org/4.x/d4/d94/tutorial_camera_calibration.html 
  * https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html 
  
* Para fazer em dupla.

* Escolha uma câmera web de baixa qualidade, execute a calibração.

* No VRI temos um tabuleiro quadriculado grande, se quiser utilizar. 

* Experimente obter as matrizes e também realizar a remoção da distorção provocada pela câmera.

* Se você achar o assunto interessante, pode calibrar duas câmeras em configuração estéreo e determinar a localização no espaço de um ponto presente em ambas imagens. 

* Submeta relatório sucinto e link para o git.

* **Data de Entrega:** 01/05

\\

---



### Download das Imagens
* OpenCV

In [1]:
%%capture
!wget https://github.com/kushalvyas/CameraCalibration/archive/refs/heads/master.zip
!unzip -o /content/master.zip

* Webcam Leboss e Newlink

In [2]:
%%capture
!wget -q --show-progress https://github.com/glenda-train/ativ_02_visao_e_percepcao/archive/refs/heads/main.zip
!unzip -o main.zip

### Download do Jupyter Slider

In [3]:
!pip install -q ipywidgets

### Importações de Bibliotecas

In [4]:
# Libs básicas de processamento de imagens e plots
import os
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

# Libs para o uso do Jupyter Slider

## Importa todas as interações
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Importa a lib de display de uma imagem
import os
from IPython.display import Image

### Definições dos Caminhos das Imagens


In [5]:
ROOT_DIRS = {
    "opencv": "/content/CameraCalibration-master/data",
    "leboss": "ativ_02_visao_e_percepcao-main/leboss_webcam",
    "newlink": "ativ_02_visao_e_percepcao-main/newlink_webcam"
}

### Funções

#### Leitura das Imagens
* OpenCV

In [6]:
# -------------------------------------------------------------------------------------------------------------

# Função para realizar a leitura das imagens (OpenCV)
## Retorna as imagens e os nomes das imagens
def read_opencv_images(root_dir, show=None):

  # Descobre o caminho para cada imagem, ignorando os diretórios
  image_paths = sorted([f for f in os.listdir(root_dir) if not os.path.isdir(os.path.join(root_dir, f))])

  # Lê todas as imagens em escala de cinza
  data = {"filenames": [], "images": []}
  for path in image_paths:
    image = cv2.imread(os.path.join(root_dir, path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    data["filenames"].append(path)
    data["images"].append(image)

  if(show is not None):
    print("Conjunto de Imagens OpenCV:")
    print("  - Quantidade de Imagens: {}".format(len(data["images"])))
    print("  - Tamanho de Imagem <altura, largura>: {}\n".format(data["images"][0].shape))

  return(data)
# -------------------------------------------------------------------------------------------------------------

* Leboss ou Newlink

In [7]:
# -------------------------------------------------------------------------------------------------------------

# Função para realizar a leitura das imagens das webcams (Leboss ou Newlink)
## Retorna as imagens e os nomes das imagens
def read_webcam_images(root_dir, show=None):
  data = {"filenames": [], "images": []}

  # Descobre o caminho das imagens
  image_paths = os.listdir(root_dir)
  image_paths = sorted(image_paths, key=lambda x: int(x[3:-4]))
  
  for path in image_paths:

    # Lê a imagem
    full_path = os.path.join(root_dir, path)
    image = cv2.imread(full_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Remover apos testes
    # image = cv2.GaussianBlur(image, ksize=(5, 5), sigmaX=0)

    # Armazena o nome dos arquivos e as imagens no dicionário
    data["filenames"].append(path)
    data["images"].append(image)

  if(show is not None):
    print("Conjunto de Imagens {}:".format(show[0].upper() + show[1:]))
    print("  - Quantidade de Imagens: {}".format(len(data["images"])))
    print("  - Tamanho de Imagem <altura, largura>: {}\n".format(data["images"][0].shape))

  return(data)
# -------------------------------------------------------------------------------------------------------------

#### Busca pelos Cantos do Tabuleiro
* Pontos de Objeto
* Pontos de Imagem

In [8]:
# -------------------------------------------------------------------------------------------------------------

# Função que encontra os pontos de canto do tabuleiro de xadrez
## Retorna os pontos do objeto, da imagem e as imagens para plotar
def find_chessboard_corners(data, chessboard_size, win_size=(11, 11), chessboard_squares_mm=1, verbose=0):

  # Define o critério padrão da OpenCV
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

  # Define os pontos inicialmente todos em zero [[0, 0, 0], ..., [0, 0, 0]]
  object_points = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)

  # Muda somente as 2 primeiras dimensões, a última fica em 0
  object_points[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)

  # Multiplica pelo tamanho do quadrado do tabuleiro em mm
  object_points *= chessboard_squares_mm

  # Pontos 3D no espaço do mundo real (de todas as imagens)
  all_object_points = []

  # Pontos 2D no plano da imagem (de todas as imagens)
  all_image_points = []

  # Conjunto de imagens para serem plotadas posteriormente
  to_plot = {"filenames": [], "images": []}

  for index, image in enumerate(data["images"]):

    # Encontra os cantos do tabuleiro
    image_path = data["filenames"][index]
    found, corners = cv2.findChessboardCorners(image, chessboard_size, None)

    # Se encontrou, refina os da imagem e armazena em uma lista
    if(found):
      all_object_points.append(object_points)
      refined_corners = cv2.cornerSubPix(image, corners, win_size, (-1, -1), criteria)
      all_image_points.append(refined_corners)

      # Faz as marcações dos cantos nas imagens e armazena em uma lista
      ## Converte para 3 canais (porque as marcações são coloridas)
      image_to_plot = cv2.merge([image, image, image])
      cv2.drawChessboardCorners(image_to_plot, chessboard_size, refined_corners, found)
      to_plot["filenames"].append(image_path)
      to_plot["images"].append(image_to_plot)

    # Se não encontrou, informa erro
    else:
      if(verbose):
        print("  - Erro: não encontrou os cantos da imagem {}".format(image_path))

  # Guarda os dados em um dicionário
  corner_data = {
      "object_points": all_object_points,
      "image_points": all_image_points,
      "to_plot": to_plot
  }
  return(corner_data)
# -------------------------------------------------------------------------------------------------------------

#### Calibração da Câmera

In [9]:
# ------------------------------------------------------------------------------------------------------------------------------------------

# Função que encontra os parâmetros de calibração da câmera, dado os pontos de objeto e pontos de imagem
def get_camera_calibration_params(object_points, image_points, image_shape, verbose=False):

  # Define o tamanho da imagem (altura e largura)
  height = image_shape[0]
  width = image_shape[1]

  # Calibração da câmera
  ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, (width, height), None, None)

  # Salva os parâmetros em um dicionário
  params = {
    "return_val": ret,                        # Valor de retorno para saber se a calibração deu certo
    "camera_matrix": mtx,                     # Matriz intrínseca da câmera
    "lens_distortion": dist,                  # Coeficientes de distorção da câmera
    "rotation_vecs": rvecs,                   # Vetores de rotação (direção e ângulo)
    "translation_vecs": tvecs                 # Vetores de translação
  }

  if(verbose):
    print("Calibração da Câmera: ", params["return_val"])
    print("\nMatriz da Câmera:\n", params["camera_matrix"])
    print("\nDistorções da Lente:\n", params["lens_distortion"])
    print("\nVetores de Rotação:\n", params["rotation_vecs"])
    print("\nVetores de Translação:\n", params["translation_vecs"])

  return(params)
# ------------------------------------------------------------------------------------------------------------------------------------------

# Função que refine os parâmetros de calibração da câmera usando uma imagem
def refine_camera_matrix(image, params):

  # Define o tamanho da imagem (altura e largura)
  height = image.shape[0]
  width = image.shape[1]

  # Gera nova matriz da câmera
  ## Retorna também a região de interesse (roi) para remover as regiões em preto perdidas pela rotação e translação
  refined_cam_matrix, roi = cv2.getOptimalNewCameraMatrix(params["camera_matrix"],
                                                          params["lens_distortion"], 
                                                          (width, height), 1, (width, height))
  
  return(refined_cam_matrix, roi)
# ------------------------------------------------------------------------------------------------------------------------------------------

#### Ajuste de Distorção
* Normal e com *Remapping*



In [10]:
# ------------------------------------------------------------------------------------------------------------------------------------------

# Função que remove a distorção de uma imagem (com método normal ou remapping)
def undistort_one_image(image, params, method, refined_cam_matrix=None, roi=None):
  height = image.shape[0]
  width = image.shape[1]

  # Aplica o método normal
  if(method == "normal"):
    
    # Remove a distorção
    undistorted_image = cv2.undistort(image, params["camera_matrix"], params["lens_distortion"], None, refined_cam_matrix)

  # Aplica o método de remapping
  elif(method == "remapping"):

    # Remove a distorção
    mapx, mapy = cv2.initUndistortRectifyMap(params["camera_matrix"], params["lens_distortion"], None, refined_cam_matrix, (width, height), 5)
    undistorted_image = cv2.remap(image, mapx, mapy, cv2.INTER_LINEAR)

  else:
    print("Erro: método não implementado")
  
  # Corta a imagem (para omitir as partes pretas inseridas pela rotação e translação)
  if(roi is not None):
    x, y, w, h = roi
    undistorted_image = undistorted_image[y:y+h, x:x+w]

  return(undistorted_image) 
# ------------------------------------------------------------------------------------------------------------------------------------------

# Função que aplica o método completo para remover a distorção de todas as imagens
def undistort_images(data, chessboard_size, method, win_size=(11, 11), chessboard_squares_mm=1, verbose=0):
  undistorted = {"images": [], "filenames": []}

  # Encontra os pontos de canto de tabuleiro de xadrez nas imagens
  corner_data = find_chessboard_corners(data, chessboard_size, win_size=win_size, chessboard_squares_mm=chessboard_squares_mm, verbose=verbose)

  # Encontra os parâmetros da calibração da câmera
  params = get_camera_calibration_params(corner_data["object_points"],
                                         corner_data["image_points"],
                                         data["images"][0].shape)
  
  # Para todas as imagens
  for index, image in enumerate(data["images"]):

    # Refina os parâmetros da câmera
    refined_cam_matrix, roi = refine_camera_matrix(image, params)

    # Remove a distorção da imagem
    undistorted_image = undistort_one_image(image, params, method, 
                                            refined_cam_matrix=refined_cam_matrix,
                                            roi=roi)
    undistorted["images"].append(undistorted_image)
    undistorted["filenames"].append(data["filenames"][index])

  return(undistorted, corner_data, params)
# ------------------------------------------------------------------------------------------------------------------------------------------

#### Erro de Reprojeção

In [11]:
# ------------------------------------------------------------------------------------------------------------------------------------------

# Calcula o erro médio da câmera
def mean_error(corner_data, params):
  mean_error = 0

  # Calcula o número de pontos de objetos
  number_of_obj_points = len(corner_data["object_points"])

  for i in range(number_of_obj_points):
    
    # Transforma ponto do objeto em ponto da imagem
    imgpoints2, _ = cv2.projectPoints(corner_data["object_points"][i],
                                      params["rotation_vecs"][i],
                                      params["translation_vecs"][i],
                                      params["camera_matrix"],
                                      params["lens_distortion"])
    
    # Calcula erro utilizando Norma Absoluta entre o que obtivemos
    # com a tranfomação e o algoritmo de localizar cantos 
    error = cv2.norm(corner_data["image_points"][i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error

  # Divide pelo número de pontos dos objetos
  mean_error /= number_of_obj_points

  return(mean_error)
# ------------------------------------------------------------------------------------------------------------------------------------------

### Main - Todas as Etapas

In [12]:
print("MAIN:")

# Lê os dados
opencv_data = read_opencv_images(ROOT_DIRS["opencv"])
leboss_data = read_webcam_images(ROOT_DIRS["leboss"])
newlink_data = read_webcam_images(ROOT_DIRS["newlink"])

# Define dicionários para guardar os dois tipos de distorção
opencv_undistorted_data = {}
leboss_undistorted_data = {}
newlink_undistorted_data = {}

# Aplica a técnica de remoção de distorção normal
opencv_emp_params = {"chessboard_size": (7, 6), "win_size": (11, 11)}
leboss_emp_params = {"chessboard_size": (4, 4), "win_size": (15, 15)}
newlink_emp_params = {"chessboard_size": (4, 7), "win_size": (3, 3)}

## OpenCV
print("\nEncontrando os Cantos do Tabuleiro nas Imagens da OpenCV:")
opencv_undistorted_data["normal"], opencv_corner_data, opencv_params = undistort_images(opencv_data,
                                                                                        opencv_emp_params["chessboard_size"],
                                                                                        "normal",
                                                                                        win_size=opencv_emp_params["win_size"],
                                                                                        verbose=1)

## Câmera Leboss
print("\nEncontrando os Cantos do Tabuleiro nas Imagens da Câmera Leboss:")
leboss_undistorted_data["normal"], leboss_corner_data, leboss_params = undistort_images(leboss_data,
                                                                                        leboss_emp_params["chessboard_size"],
                                                                                        "normal", 
                                                                                        win_size=leboss_emp_params["win_size"],
                                                                                        verbose=1)

## Câmera Newlink
print("\nEncontrando os Cantos do Tabuleiro nas Imagens da Câmera Newlink:")
newlink_undistorted_data["normal"], newlink_corner_data, newlink_params = undistort_images(newlink_data, 
                                                                                           newlink_emp_params["chessboard_size"],
                                                                                           "normal", 
                                                                                           win_size=newlink_emp_params["win_size"],
                                                                                           verbose=1)

# Aplica a técnica de remoção de distorção com remapping
opencv_undistorted_data["remapping"], _, _ = undistort_images(opencv_data,
                                                              opencv_emp_params["chessboard_size"],
                                                              "remapping", 
                                                              win_size=opencv_emp_params["win_size"])

leboss_undistorted_data["remapping"], _, _ = undistort_images(leboss_data,
                                                              leboss_emp_params["chessboard_size"],
                                                              "remapping",
                                                              win_size=leboss_emp_params["win_size"])

newlink_undistorted_data["remapping"], _, _ = undistort_images(newlink_data,
                                                               newlink_emp_params["chessboard_size"],
                                                               "remapping",
                                                               win_size=newlink_emp_params["win_size"])

# Calcula o erro
opencv_error = mean_error(opencv_corner_data, opencv_params)
leboss_error = mean_error(leboss_corner_data, leboss_params)
newlink_error = mean_error(newlink_corner_data, newlink_params)

print("\nOpenCV:")
print("  - Erro de Reprojeção: {:.4}".format(opencv_error))

print("\nCâmera Leboss:")
print("  - Erro de Reprojeção: {:.4}".format(leboss_error))

print("\nCâmera Newlink:")
print("  - Erro de Reprojeção: {:.4}".format(newlink_error))

MAIN:

Encontrando os Cantos do Tabuleiro nas Imagens da OpenCV:
  - Erro: não encontrou os cantos da imagem left09.jpg
  - Erro: não encontrou os cantos da imagem left11.jpg

Encontrando os Cantos do Tabuleiro nas Imagens da Câmera Leboss:
  - Erro: não encontrou os cantos da imagem img0.png
  - Erro: não encontrou os cantos da imagem img1.png
  - Erro: não encontrou os cantos da imagem img3.png
  - Erro: não encontrou os cantos da imagem img7.png
  - Erro: não encontrou os cantos da imagem img8.png
  - Erro: não encontrou os cantos da imagem img10.png
  - Erro: não encontrou os cantos da imagem img14.png
  - Erro: não encontrou os cantos da imagem img16.png
  - Erro: não encontrou os cantos da imagem img20.png
  - Erro: não encontrou os cantos da imagem img22.png

Encontrando os Cantos do Tabuleiro nas Imagens da Câmera Newlink:
  - Erro: não encontrou os cantos da imagem img0.png
  - Erro: não encontrou os cantos da imagem img2.png
  - Erro: não encontrou os cantos da imagem img4.pn

### Plots
#### Matplolib

In [13]:
# ------------------------------------------------------------------------------------------------------------------------------------------

# Função para plotar as imagens do conjunto de dados
def plot_dataset(data, title="", num_rows=4, fig_size=(15 , 10)):
  images = data["images"]
  filenames = data["filenames"]

  # Definições do plot
  ## Número de colunas no plot
  num_images = len(images)
  is_divisible = num_images % num_rows
  num_cols = (num_images // num_rows) + 1 if is_divisible else num_images // num_rows

  ## Criação do plot
  subplot_config = (num_rows, num_cols)

  plt.rcParams['figure.figsize'] = list(fig_size)
  figure, axes = plt.subplots(subplot_config[0], subplot_config[1])
  figure.subplots_adjust(top=0.98)

  # Plota todas as imagens lidas
  for index in range(len(images)):

    # Calcula a posição da imagem no plot
    row = math.floor(index / subplot_config[1])
    col = index % subplot_config[1]
    
    # Ajusta as informações de cada subplot
    axes[row, col].imshow(images[index], cmap="gray")
    axes[row, col].axis('off')
    axes[row, col].set_title(filenames[index])

  # Deleta os plots que não foram associados a imagens
  ids_to_remove = (num_rows * num_cols) - num_images
  for i in range(ids_to_remove):
    figure.delaxes(axes[-1, -(i+1)])

  plt.suptitle(title)
  plt.show()
# ------------------------------------------------------------------------------------------------------------------------------------------

# Plot de comparação entre 3 imagens
def plot_comparison(image_1, image_2, image_3, title="", fig_size=(15, 8)):

  # Deixa a imagem 2 com o tamanho do imagem 1
  image_2_resize = cv2.resize(image_2, (image_1.shape[1], image_1.shape[0]))

  # Deixa a imagem 3 com o tamanho do imagem 1
  image_3_resize = cv2.resize(image_3, (image_1.shape[1], image_1.shape[0]))
  
  ## Criação do plot
  subplot_config = (1, 3)
  plt.rcParams['figure.figsize'] = list(fig_size)
  figure, axes = plt.subplots(subplot_config[0], subplot_config[1])
  figure.subplots_adjust(top=0.80)

  # Ajusta as informações de cada subplot
  axes[0].imshow(image_2_resize, cmap="gray")
  axes[0].axis('off')
  axes[0].set_title("Imagem com Distorção Removida (Normal)")

  axes[1].imshow(image_1, cmap="gray")
  axes[1].axis('off')
  axes[1].set_title("Imagem Original")

  axes[2].imshow(image_3_resize, cmap="gray")
  axes[2].axis('off')
  axes[2].set_title("Imagem com Distorção Removida (Remapping)")

  plt.suptitle(title)
  plt.show()
# ------------------------------------------------------------------------------------------------------------------------------------------

# Plot de comparação entre 3 imagens
def plot_comparison_2_images(image_1, image_2, title="", fig_size=(15, 8)):

  # Deixa a imagem 2 com o tamanho do imagem 1
  image_2_resize = cv2.resize(image_2, (image_1.shape[1], image_1.shape[0]))

  ## Criação do plot
  subplot_config = (1, 2)
  plt.rcParams['figure.figsize'] = list(fig_size)
  figure, axes = plt.subplots(subplot_config[0], subplot_config[1])
  figure.subplots_adjust(top=0.80)

  # Ajusta as informações de cada subpl
  axes[0].imshow(image_1, cmap="gray")
  axes[0].axis('off')
  axes[0].set_title("Imagem Original")

  axes[1].imshow(image_2_resize, cmap="gray")
  axes[1].axis('off')
  axes[1].set_title("Imagem com Distorção Removida (Normal)")

  plt.suptitle(title)
  plt.show()
# ------------------------------------------------------------------------------------------------------------------------------------------

#### Jupyter Slider
* Conjunto de Dados

In [20]:
# Junta todos os dados
all_data = {
    "OpenCV": opencv_data,
    "Leboss": leboss_data,
    "Newlink": newlink_data,
}

# Define a barra de escolha da câmera (origem dos dados)
camera_widget = widgets.Dropdown(
    options=list(all_data.keys()),
    value=None,
    description='Dado:'
)

# Define a barra de escolha do arquivo referente à câmera
file_widget = widgets.Dropdown(
    options=[],
    value=None,
    description='Arquivo:'
)
# -------------------------------------------------------------------------------------

# Função que atualiza as escolhas de arquivos de acordo com o dado indicado
def update_files(*args):
    file_widget.options = all_data[camera_widget.value]["filenames"]
# -------------------------------------------------------------------------------------

# Função para mostrar a imagem de acordo com o nome dela
def show_image(camera, filename):
  plt.rcParams['figure.figsize'] = [6, 6]
  if(camera is None): 
    return

  # Busca o índice da arquivo indicado
  paths = all_data[camera]["filenames"]
  file_index = paths.index(filename)

  # Plota a imagem
  plt.imshow(all_data[camera]["images"][file_index], cmap='gray')
  plt.axis('off')
  plt.title("Conjunto de Dados: Arquivo {} Pertencente as Imagens da {}".format(filename, camera))
  plt.show()
# -------------------------------------------------------------------------------------

# Função que observa mudanças na seleção da câmera
camera_widget.observe(update_files, 'value')

# Função de interação
_ = interact(show_image, camera=camera_widget, filename=file_widget)
# -------------------------------------------------------------------------------------

interactive(children=(Dropdown(description='Dado:', options=('OpenCV', 'Leboss', 'Newlink'), value=None), Drop…

* Cantos Encontrados do Tabuleiro

In [21]:
# Junta todos os dados
all_data = {
    "OpenCV": opencv_corner_data["to_plot"],
    "Leboss": leboss_corner_data["to_plot"],
    "Newlink": newlink_corner_data["to_plot"],
}

# Define a barra de escolha da câmera (origem dos dados)
camera_widget = widgets.Dropdown(
    options=list(all_data.keys()),
    value=None,
    description='Dado:'
)

# Define a barra de escolha do arquivo referente à câmera
file_widget = widgets.Dropdown(
    options=[],
    value=None,
    description='Arquivo:'
)
# -------------------------------------------------------------------------------------

# Função que atualiza as escolhas de arquivos de acordo com o dado indicado
def update_files(*args):
    file_widget.options = all_data[camera_widget.value]["filenames"]
# -------------------------------------------------------------------------------------

# Função para mostrar a imagem de acordo com o nome dela
def show_image(camera, filename):
  plt.rcParams['figure.figsize'] = [6, 6]
  if(camera is None): 
    return

  # Busca o índice da arquivo indicado
  paths = all_data[camera]["filenames"]
  file_index = paths.index(filename)

  # Plota a imagem
  plt.imshow(all_data[camera]["images"][file_index], cmap='gray')
  plt.axis('off')
  plt.title("Cantos Encontrados no Tabuleiro: Arquivo {} Pertencente as Imagens da {}".format(filename, camera))
  plt.show()
# -------------------------------------------------------------------------------------

# Função que observa mudanças na seleção da câmera
camera_widget.observe(update_files, 'value')

# Função de interação
_ = interact(show_image, camera=camera_widget, filename=file_widget)
# -------------------------------------------------------------------------------------

interactive(children=(Dropdown(description='Dado:', options=('OpenCV', 'Leboss', 'Newlink'), value=None), Drop…

* Remoção da Distorção com o Método Normal

In [22]:
# Junta todos os dados
original_data = {
    "OpenCV": opencv_data,
    "Leboss": leboss_data,
    "Newlink": newlink_data,
}

normal_undistorted_data = {
    "OpenCV": opencv_undistorted_data["normal"],
    "Leboss": leboss_undistorted_data["normal"],
    "Newlink": newlink_undistorted_data["normal"],
}

remapping_undistorted_data = {
    "OpenCV": opencv_undistorted_data["remapping"],
    "Leboss": leboss_undistorted_data["remapping"],
    "Newlink": newlink_undistorted_data["remapping"],
}

# Define a barra de escolha da câmera (origem dos dados)
camera_widget = widgets.Dropdown(
    options=list(original_data.keys()),
    value=None,
    description='Dado:'
)

# Define a barra de escolha do arquivo referente à câmera
file_widget = widgets.Dropdown(
    options=[],
    value=None,
    description='Arquivo:'
)
# -------------------------------------------------------------------------------------

# Função que atualiza as escolhas de arquivos de acordo com o dado indicado
def update_files(*args):
    file_widget.options = original_data[camera_widget.value]["filenames"]
# -------------------------------------------------------------------------------------

# Função para mostrar a imagem de acordo com o nome dela
def show_image(camera, filename):
  if(camera is None): 
    return

  # Busca o índice da arquivo indicado
  paths = original_data[camera]["filenames"]
  file_index = paths.index(filename)

  # Plota a comparação das imagens
  plot_comparison(original_data[camera]["images"][file_index], 
                  normal_undistorted_data[camera]["images"][file_index],
                  remapping_undistorted_data[camera]["images"][file_index],
                  title="Comparação entre a Imagem Original e as Imagens com a Distorção Removida pelos Métodos Normal e Remapping",
                  fig_size=(20, 5))
# -------------------------------------------------------------------------------------

# Função que observa mudanças na seleção da câmera
camera_widget.observe(update_files, 'value')

# Função de interação
_ = interact(show_image, camera=camera_widget, filename=file_widget)
# -------------------------------------------------------------------------------------

interactive(children=(Dropdown(description='Dado:', options=('OpenCV', 'Leboss', 'Newlink'), value=None), Drop…

### Jupyter Slider para Avaliar os Valores de Parâmetros
* Tamanho/Padrão do Tabuleiro 
* Tamanho da Janela de Refinamento dos Cantos

In [23]:
# -------------------------------------------------------------------------------------

# Lê os dados
opencv_data = read_opencv_images(ROOT_DIRS["opencv"])
leboss_data = read_webcam_images(ROOT_DIRS["leboss"])
newlink_data = read_webcam_images(ROOT_DIRS["newlink"])
# -------------------------------------------------------------------------------------

# Função que aplica o método normal de remoção da distorção
def apply_method(data, chessboard_size, win_size):

  # Aplica a técnica de remoção de distorção normal
  undistorted_data, corner_data, params = undistort_images(data, chessboard_size, "normal", win_size=win_size, verbose=0)
  
  # Calcula o erro
  error = mean_error(corner_data, params)  
  print("Erro de Reprojeção: {:.4}".format(error))

  return(undistorted_data)
# -------------------------------------------------------------------------------------

In [24]:
# -------------------------------------------------------------------------------------

# Função que gera tuplas do número mínimo ao máximo (step de 1 em 1)
def tuples_generator(min_i, min_j, max_i, max_j,step=1):
  # Input
  #max_i, max_j = (11, 11)

  # Build sequence arrays 0, 1, ... N
  arr_i = np.arange(min_i, max_i, step)
  arr_j = np.arange(min_j, max_j, step)

  # Build cartesian product of sequence arrays
  grid = np.meshgrid(arr_i, arr_j)
  cartprod = np.stack(grid, axis=-1).reshape(-1, 2)

  # Convert to list of tuples
  result = list(map(tuple, cartprod))

  result = sorted(result, key=lambda a: a[0])  
  result = [str(tup) for tup in result]

  return(result)
# -------------------------------------------------------------------------------------

In [25]:
# ESCOLHA O DADO AQUI <opencv_data, leboss_data, newlink_data>
data = newlink_data

# Define as opções de janela e tamanho do tabuleiro
win_size_opts = tuples_generator(1, 1, 31, 31, 2)
chessboard_size_opts = tuples_generator(3, 3, 8, 8)

# Define o tamanho da janela
win_size_widget = widgets.Dropdown(
    options=win_size_opts,
    value=None,
    description='Janela:'
)

# Define o tamanho do tabuleiro
chessboard_widget = widgets.Dropdown(
    options=chessboard_size_opts,
    value=None,
    description='Tabuleiro:'
)

# Define a barra de escolha do arquivo referente à câmera
file_widget = widgets.Dropdown(
    options=data["filenames"],
    value=data["filenames"][0],
    description='Arquivo:'
)
# -------------------------------------------------------------------------------------

# Função para comparar 2 imagens
def show_image(win_size, chessboard_size, filename):

  if(win_size is None or chessboard_size is None):
    return

  # Tranforma a tupla do tamanho da janela em int
  win_size_elems = win_size[1:-1].split(",")
  int_win_size = (int(win_size_elems[0]), int(win_size_elems[1]))
  
  # Tranforma a tupla do tamanho do tabuleiro em int
  chessboard_size_elems = chessboard_size[1:-1].split(",")
  int_chessboard_size = (int(chessboard_size_elems[0]), int(chessboard_size_elems[1]))

  # Aplica o método
  print()
  undist_data = apply_method(data, int_chessboard_size, int_win_size)
  print()

  # Busca o índice da arquivo indicado
  paths = data["filenames"]
  file_index = paths.index(filename)

  # Plota a comparação das imagens
  plot_comparison_2_images(data["images"][file_index],
                           undist_data["images"][file_index],
                           title="Comparação Entre a Imagem Original e a Imagem com a Distorção Removida")
# -------------------------------------------------------------------------------------

# Função de interação
_ = interact(show_image, win_size=win_size_widget, chessboard_size=chessboard_widget, filename=file_widget)
# # -------------------------------------------------------------------------------------

interactive(children=(Dropdown(description='Janela:', options=('(1, 1)', '(1, 3)', '(1, 5)', '(1, 7)', '(1, 9)…